# Generador aleatorio de Laberintos

### Algoritmo de busqueda aleatoria en profundidad

Considere una cuadricula de celdas en la que cada celda comienza con cuatro paredes. A partir de una celda aleatoria, la computadora selecciona una celda vecina aleatoria que aún no ha sido visitada. La computadora elimina la pared entre las dos celdas y marca la nueva celda como visitada, y la agrega a la pila para facilitar el seguimiento. La computadora continúa este proceso, y una celda que no tiene vecinos no visitados se considera un callejón sin salida. Cuando se encuentra en un callejón sin salida, retrocede por el camino hasta que llega a una celda con un vecino no visitado, continuando la generación del camino al visitar esta nueva celda no visitada (creando un nuevo cruce). Este proceso continúa hasta que se han visitado todas las celdas, lo que hace que la computadora retroceda hasta la celda inicial. Podemos estar seguros de que cada celda es visitada. 

### Implementación iterativa

    Elija la celda inicial, marque la visita y presione la pila
    Mientras que la pila no está vacía
        Abre una celda de la pila y haz que sea una célula actual
        Si la celda actual tiene vecinos que no han sido visitados
            Empuja la celda actual a la pila
            Elige a uno de los vecinos no visitados
            Quitar la pared entre la celda actual y la celda elegida
            Marcar la celda elegida como visitada y empujarla a la pila

### Codigo

Importación de las librerias

In [1]:
import pygame
import sys
import random
from PIL import Image

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


<b>Render:</b> dibuja la casilla 

<b>Marcas:</b> dibuja las marcas del comiento y el final del laberinto

<b>drawBorderLine:</b>dibuja el borde entre dos puntos

<b>check_vecino:</b>comprueba las caillas vecinas no visitadas y devuelve una aleatoria

<b>indice:</b>devuelve las coordenadas de la casilla exepto si est fuera de los limites

<b>cursor:</b>dibuja la posicion de la casilla actual

In [ ]:
class Casilla:
    def __init__(self, game, x, y, TX, TY):
        self.game = game
        
        self.x = x
        self.y = y
        self.TX = TX
        self.TY = TY
        
        self.corners = [
            (x * TX, y * TY),
            (x * TX + TX, y * TY),
            (x * TX + TX, y * TY + TY),
            (x * TX, y * TY + TY),
            (x * TX, y * TY)
        ]
        
        self.walls = [True, True, True, True]
        self.visited = False
        
    def render(self):
        if self.visited:
            visitedColor = pygame.draw.rect(self.game.pantalla, self.game.VisitCOLOR, 
                (self.x * self.TX, self.y * self.TY, self.TX, self.TY))
            
        for i in range(len(self.walls)):
            if self.walls[i]:
                self.drawBorderLine(self.corners[i][0], self.corners[i][1],
                    self.corners[i + 1][0], self.corners[i + 1][1])
                
    def marcas(self):
        if self.x == 0 and self.y == 0:
            marcaInicio = pygame.draw.rect(self.game.pantalla, self.game.endCOLOR, 
                (self.x * self.TX, self.y * self.TY, self.TX, self.TY))
        
        if self.x == self.game.COLUMNAS - 1 and self.y == self.game.FILAS - 1:
            marcaFinal = pygame.draw.rect(self.game.pantalla, self.game.endCOLOR, 
                (self.x * self.TX, self.y * self.TY, self.TX, self.TY))
                
    def drawBorderLine(self, x1, y1, x2, y2):
        borderLine = pygame.draw.line(self.game.pantalla, self.game.WallCOLOR,
            (x1, y1), (x2, y2))
        
    def check_vecinos(self):
        x = self.x
        y = self.y
        
        posicionVecinos = [
            (x, y + 1),
            (x, y - 1),
            (x + 1, y),
            (x - 1, y)
        ]
        
        vecinos = []
        
        for i in range(len(posicionVecinos)):
            checkIndice = self.indice(posicionVecinos[i][0], posicionVecinos[i][1])
            
            if checkIndice is not None:
                vecinoEnCheckeo = self.game.casillas[checkIndice]
                
                if not vecinoEnCheckeo.visited:
                    vecinos.append(vecinoEnCheckeo)
                    
        if len(vecinos) > 0:
            num_rnd = random.randrange(len(vecinos))
            return vecinos[num_rnd]
        
        return None
    
    def indice(self, x, y):
        if x < 0 or x > self.game.COLUMNAS - 1 or y < 0 or y > self.game.FILAS - 1:
            return None
        
        return y * self.game.COLUMNAS + x
    
    def cursor(self):
        drawCursor = pygame.draw.rect(self.game.pantalla, self.game.CursorCOLOR,
            (self.x * self.TX, self.y * self.TY, self.TX, self.TY))

<b>inicializar dimensiones:</b>calcula el numero de filas y columnas respecto a la resolucion de la pantalla

<b>arrayObjetosCasilla:</b>crea y almacena las casillas

<b>inicializador:</b>inicializa el laberinto marcando y agregando la casilla actual

<b>abrirCamino:</b>Elimina las paredes entre la casilla actual y la casilla elegida para abrir un camino

<b>reiniciarPrograma:</b>Reinicia el laberinto

<b>printTexto:</b>imprime el texto en pantalla

<b>texto:</b>instrucciones para mostrar el texto en pantalla

<b>dibujarLaberinto:</b>dibuja todas las casillas del laberinto

<b>laberintoGenerandose:</b>algoritmo principal para la generación del laberinto, utiliza una pila para retroceder y avanzar en el laberinto

<b>actualización:</b>actualiza la pantalla

<b>chekEvent:</b>eventos del teclado

<b>guardarImagen:</b>guarda la imagen del laberinto en formato .png

<b>run:</b>bucle principal del juego que mantiene el programa en ejecución

In [ ]:
class Game:
    #colores
    def __init__(self):
        pygame.init()
        self.BackgraundCOLOR = (157, 180, 219)
        self.WallCOLOR = (35, 35, 35)
        self.CursorCOLOR = (20, 80, 220)
        self.VisitCOLOR = (108, 63, 252)
        self.PathCOLOR = (16, 170, 252)
        self.YellowCOLOR = (28, 28, 28)
        self.SpriteCOLOR = (238, 48, 16)
        self.endCOLOR = (255, 63, 63)
        
        self.TX = 50
        self.TY = 50
        
        self.RESOLUCION = (700, 700)
        self.FPS = 120
        
        self.stack = []
        self.casillas = []
        
        self.programaEjecutandose = True
        self.comenzar = False
        self.end = False
        self.elegirTamano = True
        
        self.pantalla = pygame.display.set_mode(self.RESOLUCION)
        self.reloj = pygame.time.Clock()

    def inicializarDimensiones(self):
        self.FILAS = int(self.RESOLUCION[1] / self.TY)
        self.COLUMNAS = int(self.RESOLUCION[0] / self.TX)
        
    def arrayObjetosCasilla(self):
        for i in range(self.FILAS):
            for j in range(self.COLUMNAS):
                casilla = Casilla(self, j, i, self.TX, self.TY)
                self.casillas.append(casilla)
    
    def inicializador(self):
        self.actual = self.casillas[0]
        self.actual.visitada = True
        self.stack.append(self.actual)
        
    def abrirCamino(self, actual, elegida):
        lr = actual.x - elegida.x
        
        if lr == -1:
            actual.walls[1] = False
            elegida.walls[3] = False
        elif lr == 1:
            actual.walls[3] = False
            elegida.walls[1] = False
            
        tb = actual.y - elegida.y
        
        if tb == -1:
            actual.walls[2] = False
            elegida.walls[0] = False
        elif tb == 1:
            actual.walls[0] = False
            elegida.walls[2] = False
    
    def reinicialPrograma(self):
        self.stack.clear()
        self.casillas.clear()
        self.end = False
        self.arrayObjetosCasilla()
        self.inicializador()
    
    def printTexto(self, surface, texto, size, x, y, qcolor):
        font = pygame.font.SysFont('serif', size)
        text_surface = font.render(texto, True, qcolor)
        text_rect = text_surface.get_rect()
        text_rect.center = (x, y)
        surface.blit(text_surface, text_rect)
        
    def texto(self):
        centroX = int(self.RESOLUCION[0] / 2)
        centroY = int(self.RESOLUCION[1] / 2)
        
        if self.elegirTamano:
            self.printTexto(self.pantalla, 'Selecciona el tamaño del laberinto (1-9)', 
                            int(self.RESOLUCION[0] / 25), centroX, centroY - 50, self.YellowCOLOR)
            self.printTexto(self.pantalla, 'Para guardar el laberinto presiona S al finalizar',
                            int(self.RESOLUCION[0] / 30), centroX, centroY, self.YellowCOLOR)
        elif not self.comenzar:
            self.printTexto(self.pantalla, 'Pulsa ENTER para comenzar', 
                            int(self.RESOLUCION[0] / 20), centroX, centroY, self.YellowCOLOR)
            
    def dibujarLaberinto(self):
        for i in range(len(self.casillas)):
            self.casillas[i].render()
            self.casillas[i].marcas()
    
    def laberintoGenerandose(self):
        if len(self.stack) > 0:
            self.dibujarLaberinto()
            
            self.actual = self.stack.pop()
            elegida = self.actual.check_vecinos()
            
            if elegida:
                elegida.cursor()
                self.stack.append(self.actual)
                self.abrirCamino(self.actual, elegida)
                elegida.visited = True
                self.stack.append(elegida)
            else:
                self.actual.cursor()
        else:
            self.dibujarLaberinto()
            self.end = True
    
    def actualizacion(self):
        pygame.display.set_caption(str(int(self.reloj.get_fps())))
        self.pantalla.fill(self.BackgraundCOLOR)
        
        if self.comenzar:
            self.laberintoGenerandose()
        
        self.texto()
        
        pygame.display.flip()
        self.reloj.tick(self.FPS)
        
    def checkEvent(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        
            if event.type == pygame.KEYDOWN:
                if self.elegirTamano:
                    if event.key == pygame.K_1:
                        self.TX, self.TY = 100, 100
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_2:
                        self.TX, self.TY = 70, 70
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_3:
                        self.TX, self.TY = 50, 50
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_4:
                        self.TX, self.TY = 35, 35
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_5:
                        self.TX, self.TY = 28, 28
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_6:
                        self.TX, self.TY = 25, 25
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_7:
                        self.TX, self.TY = 20, 20
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_8:
                        self.TX, self.TY = 14, 14
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_9:
                        self.TX, self.TY = 10, 10
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    
                
                elif event.key == pygame.K_RETURN:
                    if not self.comenzar:
                        self.comenzar = True
                    
                elif event.key == pygame.K_SPACE:
                    if self.comenzar:
                        self.reinicialPrograma()
                    
                elif event.key == pygame.K_s:
                    self.guardarImagen(f"laberintos/laberinto{random.randint(1, 1000)}.png")
                
                elif event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    sys.exit()
    
    def guardarImagen(self, nombre_archivo):
        pygame.image.save(self.pantalla, nombre_archivo)

    def run(self):
        while self.programaEjecutandose:
            self.checkEvent()
            self.actualizacion()
            
if __name__ == '__main__':
    game = Game()
    game.run()